# ***NoteBook ETL***

***Config***

In [ ]:
class Config:
    INPUT_PATH = 'Extract/Files/stock_senti_analysis.csv'
    SQLITE_DB_PATH = 'Extract/Files/stock_senti_analysis.db'
    SQLITE_TABLE = 'ride_bookings_clean'
    OUTPUT_PATH = 'Extract/Files/stock_senti_analysis_clean.csv'   

***Extract***

In [ ]:
class Extractor:
    """
    Clase para extraer datos de archivos fuente.
    """
    def __init__(self, file_path):
        self.file_path = file_path

    def extract(self):
        import pandas as pd


        encodings = ['utf-8', 'latin-1', 'ISO-8859-1', 'cp1252']

        for enc in encodings:
            try:
                df = pd.read_csv(self.file_path, encoding=enc)
                print(f"Archivo leído correctamente con codificación: {enc}")
                return df
            except UnicodeDecodeError:
                continue
            except Exception as e:
                print(f"Error con la codificación {enc}: {e}")
                return None

        print("No se pudo leer el archivo con ninguna codificación común.")
        return None

***Load***

In [ ]:
from Config.config import Config
import sqlite3
import os

class Loader:

    def __init__(self, df):
        self.df = df

    def to_csv(self, output_path):
        try:
            
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            self.df.to_csv(output_path, index=False)
            print(f"Datos guardados en {output_path}")
        except Exception as e:
            print(f"Error al guardar datos: {e}")

***Transformer***

In [ ]:
import pandas as pd

class Transformer:
    
    def __init__(self, df):
        self.df = df

    def clean(self):
        df = self.df.copy()

        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

        df = df.dropna(subset=['Date', 'Label'])

        df = df.drop_duplicates()
        top_cols = [col for col in df.columns if col.startswith("Top")]
        for col in top_cols:
            df[col] = (
                df[col]
                .fillna('')
                .astype(str)
                .str.strip()
                .str.replace(r'\s+', ' ', regex=True)
            )

        df['Label'] = pd.to_numeric(df['Label'], errors='coerce').fillna(0).astype(int)

        self.df = df
        return self.df

***Main***

In [ ]:
from Config.config import Config
from Extract.extractor import Extractor
from Transform.transformer import Transformer
from Load.loader import Loader
from Visualize.plots import create_plots
import os

def main():
    # Extraer datos
    extractor = Extractor(Config.INPUT_PATH)
    raw_data = extractor.extract()

    if raw_data is not None:
        # Limpiar los datos
        transformer = Transformer(raw_data)
        cleaned_data = transformer.clean()

        # Verificar que los datos limpios no están vacíos
        if cleaned_data.empty:
            print("Error: Los datos limpios están vacíos.")
            return

        # Crear la carpeta de destino para el archivo CSV si no existe
        os.makedirs(os.path.dirname(Config.OUTPUT_PATH), exist_ok=True)

        # Guardar los datos limpios en un archivo CSV
        loader = Loader(cleaned_data)
        print(f"Guardando archivo limpio en {Config.OUTPUT_PATH}")
        loader.to_csv(Config.OUTPUT_PATH)

        # Verificar si el archivo CSV se ha guardado
        if not os.path.exists(Config.OUTPUT_PATH):
            print(f"Error: No se ha encontrado el archivo CSV en {Config.OUTPUT_PATH}")
            return

        # Crear las gráficas
        print("Generando las gráficas...")
        create_plots(Config.OUTPUT_PATH)

    else:
        print("No se pudo extraer el archivo de datos.")

if __name__ == "__main__":
    main()

***Plots***

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from Config.config import Config

sns.set_palette("pastel")
sns.set_style("whitegrid")

def create_plots(output_path=None):
    output_path = output_path or Config.OUTPUT_PATH
    output_dir = "Visualize/Plots"
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(output_path):
        print(f"Error: el archivo {output_path} no existe.")
        return

    df = pd.read_csv(output_path)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])

    # - 1. Distribución de etiquetas -
    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x="Label", hue="Label", palette="pastel", legend=False)
    plt.title("Distribución de sentimientos")
    plt.xlabel("Etiqueta (0 = Negativo, 1 = Positivo)")
    plt.ylabel("Cantidad")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/01_distribucion_etiquetas.png")
    plt.close()

    # - 2. Promedio de titulares por trimestre -
    df["num_headlines"] = df[[col for col in df.columns if col.startswith("Top")]].notnull().sum(axis=1)
    headlines_per_date = df.groupby(pd.Grouper(key="Date", freq="QE"))["num_headlines"].mean().reset_index()

    plt.figure(figsize=(9, 5))
    sns.lineplot(data=headlines_per_date, x="Date", y="num_headlines", marker="o", linewidth=2.2)
    plt.title("Promedio de titulares por trimestre")
    plt.xlabel("Fecha")
    plt.ylabel("Promedio de titulares")
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/02_titulares_trimestre.png")
    plt.close()

    # - 3. Longitud promedio de titulares -
    top_cols = [col for col in df.columns if col.startswith("Top")]
    df["avg_title_length"] = df[top_cols].apply(
        lambda row: sum(len(str(x).split()) for x in row if isinstance(x, str) and x.strip()) / 
                    sum(1 for x in row if isinstance(x, str) and x.strip()) if any(isinstance(x, str) for x in row) else 0,
        axis=1
    )
    title_length_per_date = df.groupby(pd.Grouper(key="Date", freq="QE"))["avg_title_length"].mean().reset_index()

    plt.figure(figsize=(9, 5))
    sns.lineplot(data=title_length_per_date, x="Date", y="avg_title_length", marker="o", color="teal", linewidth=2.2)
    plt.title("Longitud promedio de titulares (trimestral)")
    plt.xlabel("Fecha")
    plt.ylabel("Palabras promedio por titular")
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/03_longitud_titulares.png")
    plt.close()

    # - 4. Tendencia de sentimiento -
    sentiment_trend = (
        df.groupby([pd.Grouper(key="Date", freq="QE"), "Label"]).size().reset_index(name="count")
    )
    plt.figure(figsize=(9, 5))
    sns.lineplot(data=sentiment_trend, x="Date", y="count", hue="Label", linewidth=2.2, marker="o")
    plt.title("Tendencia de sentimiento por trimestre")
    plt.xlabel("Fecha")
    plt.ylabel("Cantidad de noticias")
    plt.legend(title="Sentimiento", labels=["Negativo (0)", "Positivo (1)"])
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/04_tendencia_sentimiento.png")
    plt.close()

    # - 5. Comparación titulares vs longitud -
    merged = pd.merge(headlines_per_date, title_length_per_date, on="Date")
    plt.figure(figsize=(9, 5))
    ax1 = plt.gca()
    sns.lineplot(data=merged, x="Date", y="num_headlines", ax=ax1, color="purple", linewidth=2.2, label="Titulares")
    ax2 = ax1.twinx()
    sns.lineplot(data=merged, x="Date", y="avg_title_length", ax=ax2, color="teal", linewidth=2.2, label="Longitud")
    ax1.set_xlabel("Fecha")
    ax1.set_ylabel("Titulares promedio", color="purple")
    ax2.set_ylabel("Longitud promedio", color="teal")
    plt.title("Comparación titulares vs longitud promedio (trimestral)")
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/05_comparacion_titulares_longitud.png")
    plt.close()

    print(f"Gráficas más limpias y legibles generadas en {output_dir}")